# RChain Python gRPC client 

- This notebook is available at http://j.mp/rchain-py-grpc-walk-through

- github https://github.com/proof-media/rchain-grpc
- pypi https://pypi.org/user/proof-media/

## Introduction

### How to reproduce this demo

- Interactive locally
- Read-only mode on the web

#### Locally on your workstation 
with Docker containers

```bash
git clone https://github.com/proof-media/rchain-notebook.git
cd rchain-notebook
docker-compose up
```

Then open http://localhost:8888

In [1]:
# password is set in the `.env` file
# and shared to notebook container
%env NOTEBOOK_PASSWORD

'prooflikesrchainalot'

#### Read only mode

Through https://nbviewer.jupyter.org/

- [link to notebook](https://github.com/proof-media/rchain-notebook/blob/testing/name-registry/notebooks/walk-through.ipynb)
- [link to slides](https://nbviewer.jupyter.org/format/slides/github/proof-media/rchain-notebook/blob/testing%2Fname-registry/notebooks/walk-through.ipynb#/)

### Proof platform

![proof image](https://i0.wp.com/proofmedia.io/wp-content/uploads/2018/06/Default-Logo-copyxhdpi.png?resize=246%2C159&ssl=1)

**Because Truth Matters**

https://proofmedia.io/

> We are building a platform which leverages the wisdom of the crowds to establish truth.

- people vote in our platform on articles for being Mostly True/False
- users bet tokens to rank their conviction 
- voting closes when we reach some parameters that are proven to make "the crowd wise"
- tallying the vote will find out what the *majority* says
- we use the blockchain to lock the tokens and provide cryptographic proofs that votes weren't tampered

Rchain is needed to *register* tokens/hashes computed 

when each user votes

### Credits

Our backend is based on `Python 3.7+` and `Django framework`

- inspired by the official rchain/python snippet 

https://github.com/rchain/rchain/tree/dev/node-client


- developed mainly by Mateusz (a.k.a. `beetleman`)

https://github.com/proof-media/rchain-grpc/graphs/contributors

- I'm Paolo - CTO @Proof
- Pythonista 
- Python teacher using Jupyter Notebooks

### Roadmap and status

In [2]:
! pip search rchain | grep -i grpc

rchain-grpc (0.7.1)  - python client to rchain gRPC


* Up to version `0.0.10` -> code developed was tested with `RNode 0.6.x`
* `0.7.x` releases are compatible with `RNode 0.7.x`
* `0.8.x` releases will be compatible with `RNode 0.8.x`

- Eval/Repl [not working](https://github.com/proof-media/rchain-grpc/issues/12) for gRPC missing definition


- (near) future goal

move the repo into `rchain` GitHub space if possible, see [issue](https://github.com/proof-media/rchain-grpc/issues/8)

## Using the code

### Connecting

With current `docker-compose` for the notebook you are running a RNode *standalone* instance

In [3]:
# we have the OS NODE_NAME variable holding the other container name
RNODE_HOST = %env NODE_NAME

In [4]:
! apk add curl

/bin/sh: 1: apk: not found


In [5]:
! curl $RNODE_HOST:40403/version

RChain Node 0.7.1 (c3109d3bf326e91bf9bec7248ced8bad51e56d6b)

Now we can open a channel through `gRPC` against the RChain node 

using the Python library!

NOTE: in running docker image the package is already installed,

while normally you would do:
```bash
pip install rchain-grpc
```

In [6]:
from rchain_grpc import casper

connection = casper.create_connection(host=RNODE_HOST)
connection

### Write Rholang code

In a notebook you can write a file like this

In [7]:
%%file hello_world.rho

new print(`rho:io:stdout`) in {
    print!("Hello World!")
}


Writing hello_world.rho


Then edit it inside the notebook server,
e.g. <a href="http://localhost:8888/edit/hello_world.rho" target="_blank">link to file</a>

In [8]:
# then read the file to use the code in python
with open('hello_world.rho') as fh:
    rholang_code = fh.read()

In [9]:
# alternative: write directly the code with triple quoted strings
rholang_code = """
new print(`rho:io:stdout`) in {
    print!("Hello World!")
}
"""

In [10]:
print(rholang_code)


new print(`rho:io:stdout`) in {
    print!("Hello World!")
}



### Deploy and propose

In [11]:
casper.deploy(connection, rholang_code)

{'success': True, 'message': 'Success!'}

In [12]:
casper.propose(connection)

{'success': True, 'message': 'Success! Block 65581493c0... created and added.'}

Now check logs of the rnode to see if we have `Hello World!` string

### Play with blocks

In [13]:
output = casper.get_blocks(connection, depth=1)

# NOTE: protobuf outputs are always converted into Python dictionaries by our library
output

[{'blockHash': '65581493c0a1a47aa9f483a043b464a98f49527f6086fba4d0f0c876ccc0b5cf',
  'blockSize': '1342',
  'blockNumber': 1,
  'version': 1,
  'deployCount': 1,
  'tupleSpaceHash': '477f7f0c469de6d500d8a1f74852a4ecf49df9a7fa463207d62258161bf39fb7',
  'timestamp': 1541548724337,
  'faultTolerance': -1.0,
  'mainParentHash': '8e90e1071b1df8cb84ec0ab06847d5eb6b7e3f708a75b8e1301f3524b317abc7',
  'parentsHashList': ['8e90e1071b1df8cb84ec0ab06847d5eb6b7e3f708a75b8e1301f3524b317abc7'],
  'sender': 'eabe5a1a0750d2a8745709bb0bdb24f63c6a8ac3a887b9bed40b34b0598ddf08'}]

In [14]:
block_hash = output.pop().get('blockHash')
block_hash

'65581493c0a1a47aa9f483a043b464a98f49527f6086fba4d0f0c876ccc0b5cf'

In [15]:
block = casper.get_block(connection, block_hash=block_hash)
block

{'blockHash': '65581493c0a1a47aa9f483a043b464a98f49527f6086fba4d0f0c876ccc0b5cf',
 'blockSize': '1342',
 'blockNumber': 1,
 'version': 1,
 'deployCount': 1,
 'tupleSpaceHash': '477f7f0c469de6d500d8a1f74852a4ecf49df9a7fa463207d62258161bf39fb7',
 'tupleSpaceDump': '@{Unforgeable(0xa50a17f94a9bf43206517f07ddcd76e64e8350533ad0b6715b0e559eac10b8dd)}!(`rho:id:dputnspi15oxxnyymjrpu7rkaok3bjkiwq84z7cqcrx4ktqfpyapn4`) |\n@{Unforgeable(0x861583b3fab1a04eed7eeb4c03f53450a7cc12bd184944787b99b8f58b577a4d)}!({5122eac8a5c99b7da8f6d40d5f3269c6844dabde0418749ece2aeedc64c27703 : (288, "secp256k1Verify", Nil, 1), 5a3fff1ed432237e779fc6aa20d7549d043cc0dd92180ebee0346229598870f8 : (228, "secp256k1Verify", Nil, 2), d6b0c603a51fbcd56db1606a0591310876f93bdbf2309c7bdbc277a080368990 : (188, "secp256k1Verify", Nil, 4), a3158195b3ca6dfb88b8f0cb0788b01f25ea11421104df071bd043ca27f2def4 : (202, "secp256k1Verify", Nil, 3), e84b0ca96139b9c5e1a58b55e9d84682318a2030eaf24923204a1473ed659b34 : (224, "secp256k1Verify", Nil

In [16]:
block.get('blockNumber')

1

### Context Manager

The connection provided by the library can be used in a `with` python statement

In [17]:
# all previous operations in one context

with casper.create_connection(host=RNODE_HOST) as connection:

    # deploy / propose
    casper.deploy(connection, rholang_code)
    print(casper.propose(connection))
    
    # handle output
    output = casper.get_blocks(connection, depth=1)
    block_hash = output.pop().get('blockHash')
    block = casper.get_block(connection, block_hash=block_hash)
    print(f"Current block number is {block.get('blockNumber')}\nwith hash {block_hash}")
    
# connection here is closed

{'success': True, 'message': 'Success! Block bee1f076d5... created and added.'}
Current block number is 2
with hash bee1f076d58eb681743e3ab6376292075a9faf1abf4899fee68d799da6a1ec34


### Interact with channels

Now we'll take advantage of the `listenForDataAtName` functionality inside the RChain gRPC definition.

We can specify a channel to receive the results of the new block proposed.

In [18]:
output_placeholder = "your_channel_name"

In [19]:
rholang_code = f"""
{output_placeholder}!("bar")
""" 

In [20]:
print(rholang_code)

# your_channel_name will be replaced with @"RANDOMNAME"


your_channel_name!("bar")



In [21]:
with casper.create_connection(host=RNODE_HOST) as connection:
    block = casper.run_and_get_value_from(
        connection=connection,
        term=rholang_code, 
        # this name gets replaced with a channel name
        output_placeholder=output_placeholder
    )

In [22]:
# what's used for replacing the placeholder
import secrets
ack_name = f'ack_{secrets.token_hex(30)}'
ack_name

'ack_ff026dd2fd8a89fc12604abdbe5f4e0552a33c98a3d763b0ef10a975b724'

In [23]:
block

{'status': 'Success',
 'blockResults': [{'postBlockData': [['bar']],
   'block': {'blockHash': 'ceaeb77444283f5f19341b39d6e3527c516f80ae19b906940a4f60aa66569966',
    'blockSize': '1146',
    'blockNumber': 3,
    'version': 1,
    'deployCount': 1,
    'tupleSpaceHash': '7bf4c2b8d44b1b25debf422b49ddb8d7a75506227d486709a522d9343576bfef',
    'timestamp': 1541548733670,
    'faultTolerance': -1.0,
    'mainParentHash': 'bee1f076d58eb681743e3ab6376292075a9faf1abf4899fee68d799da6a1ec34',
    'parentsHashList': ['bee1f076d58eb681743e3ab6376292075a9faf1abf4899fee68d799da6a1ec34'],
    'sender': 'eabe5a1a0750d2a8745709bb0bdb24f63c6a8ac3a887b9bed40b34b0598ddf08'}}],
 'length': 1}

In [24]:
results = block.get('blockResults').pop()

for message in results.get('postBlockData'):
    print("Received: ", message.pop())

Received:  bar


### Name registry

`NEW!`

(see [link to specs](https://rchain.atlassian.net/wiki/spaces/RHOL/pages/511541342/Name+registry+specification))

Using the techniques tested so far we can now:

- write a contract with `unforgeable name`
- register its name to Name Registry (introduced in 0.7) 
- and get the id for lookup

so - all programmatically from Python - 

we can (almost) call a contract in a secure way.

#### 1. Register a contract

In [25]:
ack_name = 'placeholder_something'

In [26]:
rholang_code = """
new newName, ack, register(`rho:registry:insertArbitrary`), stdout(`rho:io:stdout`) in
{
  register!(bundle+{*newName}, *ack)
  |
  contract newName(@msg) = {
    stdout!("Contract called with message: " ++ msg)
  }
  |
  for (@msg <- ack) {
    %s!(["From registry: ", msg])
  }
}
""" % ack_name

In [27]:
print(rholang_code)


new newName, ack, register(`rho:registry:insertArbitrary`), stdout(`rho:io:stdout`) in
{
  register!(bundle+{*newName}, *ack)
  |
  contract newName(@msg) = {
    stdout!("Contract called with message: " ++ msg)
  }
  |
  for (@msg <- ack) {
    placeholder_something!(["From registry: ", msg])
  }
}



In [28]:
# run and get the value in the block output
with casper.create_connection(host=RNODE_HOST) as connection:
    block = casper.run_and_get_value_from(connection, rholang_code, ack_name)
    
from pprint import pprint
pprint(block)

{'blockResults': [{'block': {'blockHash': '2b87d1204918bb68e92614fcfea344a2b78b9f1957b6b3c7fea3f0ec8cac6850',
                             'blockNumber': 4,
                             'blockSize': '3187',
                             'deployCount': 1,
                             'faultTolerance': -1.0,
                             'mainParentHash': 'ceaeb77444283f5f19341b39d6e3527c516f80ae19b906940a4f60aa66569966',
                             'parentsHashList': ['ceaeb77444283f5f19341b39d6e3527c516f80ae19b906940a4f60aa66569966'],
                             'sender': 'eabe5a1a0750d2a8745709bb0bdb24f63c6a8ac3a887b9bed40b34b0598ddf08',
                             'timestamp': 1541548738908,
                             'tupleSpaceHash': '109aca205eea7930e9d8a4e9cde311ebeaeb73558c3b43d2083a559c5410ca2f',
                             'version': 1},
                   'postBlockData': [[{'ps': [['From registry: '],
                                              ['rho:id:rqjd73cs7tuadha

#### 2. WIP: get ID inside the code

We are looking to find a better way to handle the "post block data" 

In [29]:
post_block_data = block.get('blockResults').pop().get('postBlockData')

In [30]:
post_block_data

[[{'ps': [['From registry: '],
    ['rho:id:rqjd73cs7tuadhamaow7nans57mfpdz96b67gd6ma6ow95eh8zc6ej']]}]]

In [31]:
list(post_block_data[0][0].values())[0][1][0]

'rho:id:rqjd73cs7tuadhamaow7nans57mfpdz96b67gd6ma6ow95eh8zc6ej'

In [32]:
# alternative with json/regular expression

import re
import json

post_block_str = json.dumps(post_block_data)

match = re.search(r"rho:id:[^\"]+",  post_block_str)
registry_id = match.group()

In [33]:
registry_id

'rho:id:rqjd73cs7tuadhamaow7nans57mfpdz96b67gd6ma6ow95eh8zc6ej'

#### 3. Lookup and call the contract

In [34]:
rholang_code = """
new return, lookup(`rho:registry:lookup`), stdout(`rho:io:stdout`) in
{
  lookup!(`%s`, *return) |
  for (myContract <- return) {
    myContract!("Proof test")
  }
}
""" % registry_id
print(rholang_code)


new return, lookup(`rho:registry:lookup`), stdout(`rho:io:stdout`) in
{
  lookup!(`rho:id:rqjd73cs7tuadhamaow7nans57mfpdz96b67gd6ma6ow95eh8zc6ej`, *return) |
  for (myContract <- return) {
    myContract!("Proof test")
  }
}



In [35]:
with casper.create_connection(host=RNODE_HOST) as connection:
    print(casper.deploy(connection, rholang_code))
    print(casper.propose(connection))

{'success': True, 'message': 'Success!'}
{'success': True, 'message': 'Success! Block fabb9c8e19... created and added.'}


go check the logs!

# THE END